# this notebook contains steps taken to write the loss function (in particular the likelihood) for Geometric Gaussian Process LPU in terms of logsumexp function only

In [53]:

import numpy as np
import scipy.special
import torch

for trial in range(100):
    l_i = np.random.binomial(1, 0.5)
    gamma_sample=np.random.normal(0, 1)
    lambda_sample=np.random.normal(0, 1)    
    L_t = np.random.normal(0, 1)
    L_s = np.random.normal(0, 1)
    gamma_prime, lambda_prime, _ = torch.softmax(torch.tensor((gamma_sample, lambda_sample, 0.)), dim=0)

    # original term
    L_i = l_i * (np.log(scipy.special.expit(L_t)) + np.log(gamma_prime + (1 - gamma_prime - lambda_prime)*scipy.special.expit(L_s))) + \
        (1 - l_i)*(np.log(1 - scipy.special.expit(L_t)*(gamma_prime + (1 - gamma_prime - lambda_prime)*scipy.special.expit(L_s))))    

    # one level expansion
    L_i_prime = l_i * (np.log(scipy.special.expit(L_t)) + np.log((np.exp(gamma_sample)/(np.exp(gamma_sample) + np.exp(lambda_sample) + 1)) + (1 - (np.exp(gamma_sample)/(np.exp(gamma_sample) + np.exp(lambda_sample) + 1)) - (np.exp(lambda_sample)/(np.exp(gamma_sample) + np.exp(lambda_sample) + 1)))*scipy.special.expit(L_s))) + \
        (1 - l_i)*(np.log(1 - scipy.special.expit(L_t)*((np.exp(gamma_sample)/(np.exp(gamma_sample) + np.exp(lambda_sample) + 1)) + (1 - (np.exp(gamma_sample)/(np.exp(gamma_sample) + np.exp(lambda_sample) + 1)) - (np.exp(lambda_sample)/(np.exp(gamma_sample) + np.exp(lambda_sample) + 1)))*scipy.special.expit(L_s))))
    
    #full expansion
    # see https://claude.ai/chat/c6c9fec4-ee22-47d7-85ad-e54eacbce667 for details
    expanded_L_i =  l_i * (-np.log(1 + np.exp(-L_t)) 
                           + np.log(gamma_prime * (1 + np.exp(-L_s)) + (1 - gamma_prime - lambda_prime)) 
                           - np.log(1 + np.exp(-L_s))) \
                    + (1 - l_i) * (np.log((1 + np.exp(-L_t)) * (1 + np.exp(-L_s)) - (gamma_prime * (1 + np.exp(-L_s)) + (1 - gamma_prime - lambda_prime))) - np.log((1 + np.exp(-L_t)) * (1 + np.exp(-L_s))))

    L_i_prime_no_fraction = l_i * (np.log(scipy.special.expit(L_t)) + np.log(np.exp(gamma_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1) + (1 - np.exp(gamma_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1) - np.exp(lambda_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1))*scipy.special.expit(L_s))) + \
           (1 - l_i)*(np.log(1 - scipy.special.expit(L_t)*(np.exp(gamma_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1) + (1 - np.exp(gamma_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1) - np.exp(lambda_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1))*scipy.special.expit(L_s))))
    
    L_i_prime_no_fraction_no_expit = l_i * (np.log(1 / (1 + np.exp(-L_t))) + np.log(np.exp(gamma_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1) + (1 - np.exp(gamma_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1) - np.exp(lambda_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1))*(1 / (1 + np.exp(-L_s))))) + \
           (1 - l_i)*(np.log(1 - (1 / (1 + np.exp(-L_t)))*(np.exp(gamma_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1) + (1 - np.exp(gamma_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1) - np.exp(lambda_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1))*(1 / (1 + np.exp(-L_s))))))    
    # logsumexp_expanded_L_i = l_i * (scipy.special.logsumexp([-L_t, 0]) + scipy.special.logsumexp([scipy.special.logsumexp([gamma_sample - np.log(np.exp(gamma_sample) + np.exp(lambda_sample) + 1), 0]), 
    #                                                  scipy.special.logsumexp([-L_s, np.log(1 - np.exp(scipy.special.logsumexp([gamma_sample - np.log(np.exp(gamma_sample) + np.exp(lambda_sample) + 1), 0])) - 
    #                                                                          np.exp(scipy.special.logsumexp([lambda_sample - np.log(np.exp(gamma_sample) + np.exp(lambda_sample) + 1), 0])))])])) + \
    #         (1 - l_i) * (scipy.special.logsumexp([0, -scipy.special.logsumexp([-L_t, 0]) - scipy.special.logsumexp([scipy.special.logsumexp([gamma_sample - np.log(np.exp(gamma_sample) + np.exp(lambda_sample) + 1), 0]), 
    #                                                                         scipy.special.logsumexp([-L_s, np.log(1 - np.exp(scipy.special.logsumexp([gamma_sample - np.log(np.exp(gamma_sample) + np.exp(lambda_sample) + 1), 0])) - 

    L_i_prime_no_fraction_no_expit = l_i * (np.log(1 / (1 + np.exp(-L_t))) \
                                            + np.log(np.exp(gamma_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1) 
                                                   + (1 - np.exp(gamma_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1) 
                                                      - np.exp(lambda_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1))*(1 / (1 + np.exp(-L_s))))) + \
           (1 - l_i)*(np.log(1 - (1 / (1 + np.exp(-L_t)))*(np.exp(gamma_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1) + (1 - np.exp(gamma_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1) - np.exp(lambda_sample) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)**(-1))*(1 / (1 + np.exp(-L_s))))))    

    A = np.exp(gamma_sample)
    B = np.exp(lambda_sample)
    C = 1 + np.exp(-L_t)
    D = 1 + np.exp(-L_s)

    # C * D * (A + B + 1) =  (1 + np.exp(-L_t) + np.exp(-L_s) + np.exp(-L_t -L_s)) * (np.exp(gamma_sample) + np.exp(lambda_sample) + 1)
    # C * D * (A + B + 1) - A * D -1  is the "term" variable below:
    term = np.exp(gamma_sample - L_t) + np.exp(-L_t-L_s+gamma_sample) \
                             + np.exp(lambda_sample) + np.exp(lambda_sample - L_t) + np.exp(lambda_sample - L_s) + np.exp(-L_t-L_s+lambda_sample) \
                             + np.exp(-L_t) + np.exp(-L_s) + np.exp(-L_t-L_s)
    
    # writing the loglikelhood with A, B, C, D as variables
    L_i_prime_with_aux_variables = l_i * (-np.log(C) + np.log(A * D + 1) - np.log((A + B + 1) * D)) + (1 - l_i) * (np.log((term)) - np.log((C * (A + B + 1) * D)))


    L_i_prime_with_logsumexp = l_i * (-scipy.special.logsumexp([0, -L_t]) + scipy.special.logsumexp([gamma_sample, gamma_sample - L_s, 0]) - scipy.special.logsumexp([gamma_sample, + lambda_sample, 0]) - scipy.special.logsumexp([0, -L_s])) \
            + (1 - l_i) * (scipy.special.logsumexp([gamma_sample - L_t, gamma_sample - L_t - L_s, lambda_sample, lambda_sample - L_t, lambda_sample - L_s, lambda_sample - L_t - L_s,-L_t, -L_s, -L_t - L_s])\
                           - scipy.special.logsumexp([gamma_sample, gamma_sample - L_s, gamma_sample - L_t, gamma_sample - L_s - L_t, 
                                                      lambda_sample, lambda_sample - L_s, lambda_sample - L_t, lambda_sample - L_t - L_s, 0, -L_t, -L_s, -L_t - L_s]))

    logit = -scipy.special.logsumexp([0, -L_t]) + scipy.special.logsumexp([gamma_sample, gamma_sample - L_s, 0]) - scipy.special.logsumexp([gamma_sample, + lambda_sample, 0]) - scipy.special.logsumexp([0, -L_s]) \
    -scipy.special.logsumexp([gamma_sample - L_t, gamma_sample - L_t - L_s, lambda_sample, lambda_sample - L_t, lambda_sample - L_s, lambda_sample - L_t - L_s,-L_t, -L_s, -L_t - L_s])\
    +scipy.special.logsumexp([gamma_sample, gamma_sample - L_s, gamma_sample - L_t, gamma_sample - L_s - L_t, 
                                                      lambda_sample, lambda_sample - L_s, lambda_sample - L_t, lambda_sample - L_t - L_s, 0, -L_t, -L_s, -L_t - L_s])
    
    # print (np.log((C * (A + B + 1) * D - A * D - 1)), 3 * np.log(C) + np.log(B) + 2 * np.log(D))
    # L_i_prime_log_sum_exp =  l_i * (-scipy.special.logsumexp([0, L_t]) \
    #                                 + gamma_sample - scipy.special.logsumexp([gamma_sample, lambda_sample, 0])\ 
    #                                 -scipy.special.logsumexp([gamma_sample, lambda_sample, 0]) + scipy.special.logsumexp([0, L_s])])) + \
    #                              (1 - l_i) * (np.log(-np.expm1(-scipy.special.logsumexp([0, L_t]) - scipy.special.logsumexp([gamma_sample - scipy.special.logsumexp([gamma_sample, lambda_sample, 0]), -scipy.special.logsumexp([gamma_sample, lambda_sample, 0]) + scipy.special.logsumexp([0, L_s])]))))
                                                                                                        # np.exp(scipy.special.logsumexp([lambda_sample - np.log(np.exp(gamma_sample) + np.exp(lambda_sample) + 1), 0])))])])]))
    if not np.allclose(L_i, L_i_prime):
        raise ValueError(f"Error: {L_i} != {L_i_prime}")
    
    if not np.allclose(L_i, expanded_L_i, atol=1e-6):
        raise ValueError(f"Error: {L_i} != {expanded_L_i}")

    if not np.allclose(L_i, L_i_prime_no_fraction, 1e-6):
        raise ValueError(f"Error: {L_i} != {L_i_prime_no_fraction}, and the difference is {np.abs(L_i - L_i_prime_no_fraction)}")
    
    if not np.allclose(L_i, L_i_prime_no_fraction_no_expit, 1e-6):
        raise ValueError(f"Error: {L_i} != {L_i_prime_no_fraction_no_expit}, and the difference is {np.abs(L_i - L_i_prime_no_fraction_no_expit)}")

    if not np.allclose(L_i, L_i_prime_with_aux_variables, 1e-6):
        raise ValueError(f"Error: {L_i} != {L_i_prime_with_aux_variables}, and the difference is {np.abs(L_i - L_i_prime_with_aux_variables)}")

    if not np.allclose(L_i, L_i_prime_with_logsumexp, 1e-6):
        raise ValueError(f"Error: {L_i} != {L_i_prime_with_logsumexp}, and the difference is {np.abs(L_i - L_i_prime_with_logsumexp)}")

    # if not np.allclose(L_i, L_i_prime_log_sum_exp, 1e-6):
    #     raise ValueError(f"Error: {L_i} != {L_i_prime_log_sum_exp}, and the difference is {np.abs(L_i - L_i_prime_log_sum_exp)}")


In [23]:
np.log((np.exp(gamma_sample)/(np.exp(gamma_sample) + np.exp(lambda_sample) + 1)) + (1 - (np.exp(gamma_sample)/(np.exp(gamma_sample) + np.exp(lambda_sample) + 1)) - (np.exp(lambda_sample)/(np.exp(gamma_sample) + np.exp(lambda_sample) + 1)))*scipy.special.expit(L_s))

-0.7706925594594443

In [ ]:
L_i_prime